# Regime Detection Using Economic Variables

Renjie Pan (renjie.pan@nyu.edu)

Tianyu Zhang (tianyuzhang@nyu.edu)

Liang Zou (liazou@nyu.edu)

## Introduction

Economic factors usually lead to regime shifts in financial markets. In this project, we try to develop a two-state Hidden Markov Model (HMM) to detect the regime given time series economic data (financial turbulence, economic growth, and inflation) based on the article "Regime Shifts: Implications for Dynamic Strategies" (Kritzman, Page, Turkington, 2012). We will first compute equity turbulence and currency turbulence from S&P 500 sector indices and currencies versus U.S. dollar respectively. After that, we perform exploratory data analysis (EDA) on these time series and conduct data cleansing (trimming) to remove anomaly points. At the end, we use HMM to generate two regimes for each time series.

In [1]:
import numpy as np
import pandas as pd
import random
import math

## Dataset Description and Loading

The economic variables we will use are the following, where data were collected from Federal Reserve Economic Data (FRED) website:

$\textbf{Economic Growth}$: Quarter-over-quarter percentage change in seasonally adjusted U.S. real gross national product (GNP) from second quarter 1947 to fourth quarter 2009.

$\textbf{Inflation}$: Monthly percentage changes in seasonally adjuested US. Consumer Price Index (CPI) for all urban consumers from February 1947 to December 2009.

$\textbf{Sector Levels}$: 10 S&P 500 sector indices between December 1975 and December 2009.

$\textbf{Currency Pairs}$: G-10 currencies between December 1977 and December 2009.

In [2]:
directory = '/Users/liangzou/Desktop/Project-and-Presentation-Fall-2019/BNP Project/'
data_file_name = 'kritzman data.xlsx'

econ_growth = pd.read_excel(directory + data_file_name, 0)
inflation = pd.read_excel(directory + data_file_name, 1)
sector = pd.read_excel(directory + data_file_name, 2)
currency = pd.read_excel(directory + data_file_name, 3)

## Compute Financial Turbulence

We compute the financial turbulence using the following:

$$d_t = (y_t - \mu) \Sigma^{-1} (y_t - \mu)'$$

$y_t$: asset returns for period t

$\mu$: sample average historical returns

$\Sigma$: sample covariance matrix of historical returns

$\textbf{Equity Turbulence}$: Historical returns for past 10 years for S&P 500 sector indices

$\textbf{Currency Turbulence}$: Historical returns for past 3 years for G-10 currencies versus U.S. dollar (USD)

In [3]:
def compute_turbulence(df, years=3, alpha=0.01):
    '''
    Compute financial turbulence given time series data
        input: 
            df || DataFrame || a Dataframe includes Column "Date"
            years || integer || number of years to compute historical returns
            alpha || float || a punishment coefficient when inverse-coveriance is singular
        output: Turbulence || DataFrame || Column = ["Date", "Turbulence"]
    '''
    
    # Compute return for this series
    df_return = df.iloc[1:,1:].values / df.iloc[:-1,1:].values - 1
    distance = []
    error = []
    days_in_year = 252
    
    for i in range(years * days_in_year, len(df)-1):
        df_past_return = df_return[:i+1,:]
        # Compute historical mean return
        mu = np.mean(df_past_return, axis=0)
        try:
            # Compute inverse covariance matrix
            inv_sig = np.linalg.inv(np.cov(df_past_return.T))
        except:
            # Find days when covariance matrices are not invertible
            # and add small numbers to the diagonal
            sigma = np.cov(df_past_return.T)
            x = np.ones(sigma.shape[0])
            inv_sig = np.linalg.inv(sigma + np.diag(x)*alpha)
            error.append(i)

        y = np.array(df_return[i,:])
        d = np.dot(np.dot(y-mu, inv_sig),(y-mu).T)
        distance.append(d)

    Turbulence = pd.DataFrame({'Date': df['Date'][-len(distance):], 'Turbulence': distance})
    
    if error != []:
        print('Rows that produce singular covariance matrix')
        print(np.array(error) + 2)
    
    return Turbulence

In [4]:
sector_turbulence = compute_turbulence(sector, years=10)
currency_turbulence = compute_turbulence(currency, years=3)

## Exploratory Data Analysis and Data Cleansing

In [5]:
EDA_dataframe = pd.concat([econ_growth.describe(), inflation.describe(), 
           sector_turbulence.describe(), currency_turbulence.describe()], axis=1)
EDA_dataframe.columns = ['Economic Growth','Inflation','Equity Turbulence','Currency Turbulence']
EDA_dataframe.round(4)

,Economic Growth,Inflation,Equity Turbulence,Currency Turbulence
count,289.0000,870.0000,4905.0000,6669.0000
mean,3.2128,0.2847,10.6787,7.9320
std,3.8463,0.3443,18.0354,27.8651
min,-10.0000,-1.8000,0.0023,0.1927
25%,1.2000,0.1000,2.9631,2.7772
50%,3.1000,0.2000,5.2398,4.8709
75%,5.1000,0.4000,10.6382,8.7553
max,16.7000,2.0000,393.9824,2072.4420


In [6]:
currency_turbulence.loc[currency_turbulence.iloc[:,1] > 2000,:]

,Date,Turbulence
6265,2015-01-15,2072.44202


The descriptive statistics show all 4 time series are right-skewed with larger mean than median. The skewness is extremely large in equity and currency turbulence since a number of observations are greater than 100 whereas the means are 10.67 in equity and 7.93 in currency. 

On January 15th, 2015, the currency turbulence is 2072, which resulted from special event: The Swiss National Bank abandoned cap on the Swiss franc’s exchange rate against the euro.

In [7]:
def winsorize_series(s, q=0.001):
    """"
    This function returns the trimmed version of a Series.
        input:
            s | Series | Series before trimmed
            q | float  | specific quantile to trim
        output:
            s | Series | Series after trimmed
    """
    q = s.quantile([q, 1-q])
    if isinstance(q, pd.Series) and len(q) == 2:
        s[s < q.iloc[0]] = q.iloc[0]
        s[s > q.iloc[1]] = q.iloc[1]
    return s

In [8]:
econ_growth.iloc[:,1] = winsorize_series(pd.Series(econ_growth.iloc[:,1]))
inflation.iloc[:,1] = winsorize_series(pd.Series(inflation.iloc[:,1]))
sector_turbulence.loc[:,'Turbulence'] = winsorize_series(pd.Series(sector_turbulence.loc[:,'Turbulence']))
currency_turbulence.loc[:,'Turbulence'] = winsorize_series(pd.Series(currency_turbulence.loc[:,'Turbulence']))

To reduce the impact of anomaly points, we trim the time series with 0.1% - 99.9% quantile.

## HMM based on Baum-Welch Algorithm

In [9]:
def fit_hmm(y, dim=2, max_iteration=1000, tolerance=1e-6):
    """
    Fit Hidden Markov Model based on Baum-Welch Algorithm.
    Here, we assume each regime is normally distributed with certain mean and variance.
    @y: Series (time series data)
    @dim: integer (number of regimes or states)
    @max_iteration: interger (maximum number of iterations)
    @tolerance: float (tolerance for convergence)
    @return: list (A: transition probability matrix, mu: mean of normal distribution,
                    sigma: standard deviation of normal distribution, p: initial probability)
    """
    
    # random.seed(42)
    T = len(y)
    mu_y = np.mean(y)
    sigma_y = np.std(y)
    mu = [mu_y] * dim
    sigma = [sigma_y] * dim
    
    for d in range(dim): 
        mu[d] += np.asscalar(np.random.randn(1, 1) * sigma_y)
        # mu[d] += np.asscalar((0.1 * (d + 1))* sigma_y)
    
    # Initialize transition probability matrix and initial probability
    A = np.ones((dim, dim)) / dim
    p = np.ones((1, dim)) / dim
    PI = math.pi
    
    iteration = 1
    likelihood = [-2**31]
    change_likelihood = 2**31
    
    # Initialize matrices
    B = np.zeros((T, dim))
    forward = np.zeros((T, dim))
    backward = np.zeros((T, dim))
    scale = np.zeros((T, dim))
    smoothed = np.zeros((T, dim))
    xi = np.zeros((dim, dim, T))
    
    while change_likelihood > tolerance and iteration < max_iteration:
        
        for t in range(T):
            for d in range(dim):
                # Compute normal pdf for each time step, each regime
                B[t,d] = np.asscalar(1 / np.sqrt(2 * PI * sigma[d]**2) * np.exp(-0.5 * ((y[t] - mu[d])/sigma[d])**2))
        
        forward[0,:] = p * B[0,:]
        scale[0,:] = np.sum(forward[0,:])
        forward[0,:] = forward[0,:] / np.sum(forward[0,:])
        
        for t in range(1, T):
            forward[t,:] = np.dot(forward[t-1,:], A)*B[t,:]
            scale[t,:] = np.sum(forward[t,:])
            forward[t,:] = forward[t,:] / np.sum(forward[t,:])
            
        backward[T-1,:] = B[T-1,:]
        backward[T-1,:] = backward[T-1,:] / np.sum(backward[T-1,:])
        
        for t in range(T-2, -1 ,-1):
            backward[t,:] = np.dot(A, backward[t+1,:].T).T*B[t+1,:]
            backward[t,:] = backward[t,:] / np.sum(backward[t,:])
            
        for t in range(T):
            smoothed[t,:] = forward[t,:]*backward[t,:]
            smoothed[t,:] = smoothed[t,:] / np.sum(smoothed[t,:])
            
        for t in range(T-1):
            xi[:,:,t] = A * np.dot(forward[t,:].reshape((dim, 1)), (backward[t+1,:]*B[t+1,:]).reshape((1, dim)))
            xi[:,:,t] = xi[:,:,t] / np.sum(xi[:,:,t])
                
        p = smoothed[0,:]
        exp_num_transitions = np.sum(xi, axis = 2)
        
        for d in range(dim):
            A[d,:] = exp_num_transitions[d,:] / np.sum(xi[d,:,:])
            mu[d] = np.asscalar(np.dot(smoothed[:,d].T,y).T / np.sum(smoothed[:,d]))   
            sigma[d] = np.asscalar(np.sqrt(np.sum(smoothed[:,d]*(y-mu[d])*(y-mu[d])) / np.sum(smoothed[:,d])))
            
        likelihood.append(np.asscalar(np.sum(np.log(scale))))
        change_likelihood = np.asscalar(np.abs(likelihood[-1]-likelihood[-2]))
        iteration +=1
    
    print('Transition Probability =')
    print(np.round(A, 4))
    
    print('Mean =')
    print(np.round(mu, 4))
    
    print('Standard Deviation =')
    print(np.round(sigma, 4))
    
    print('Initial Probability =')
    print(np.round(p, 4))
    
    return [A, mu, sigma, p]

## Regime Detection on Economic Growth

In [10]:
econ_hmm = fit_hmm(econ_growth.iloc[:,1].values)

Transition Probability =
[[0.9738 0.0262]
 [0.0203 0.9797]]
Mean =
[3.4483 2.9489]
Standard Deviation =
[4.9376 1.8956]
Initial Probability =
[1. 0.]


Based on economic growth time series, the two regimes are normally distributed with mean 3.45, 2.95 and standard deviation 4.94, 1.90 respectively. The result shows the center of two regimes are close to each other but the first one is more volatile.

## Regime Detection on Inflation

In [11]:
inflation_hmm = fit_hmm(inflation.iloc[:,1].values)

Transition Probability =
[[0.9869 0.0131]
 [0.0458 0.9542]]
Mean =
[0.2035 0.5433]
Standard Deviation =
[0.2002 0.512 ]
Initial Probability =
[0. 1.]


In terms of inflation, the first regime has mean 0.20 with standard deviation 0.20 while the second one produces a normal distribution with mean 0.54 and standard deviation 0.51. 

## Regime Detection on Equity turbulence

In [12]:
sector_hmm = fit_hmm(sector_turbulence.iloc[:,1].values)

Transition Probability =
[[0.8917 0.1083]
 [0.0409 0.9591]]
Mean =
[27.8203  4.881 ]
Standard Deviation =
[26.35    3.2517]
Initial Probability =
[1. 0.]


The two regimes in equity turbulence are distinct to each other. The first one has mean 27.82 with standard deviation 26.35 while the later one has mean 4.88 and standard deviation 3.25.

## Regime Detection on Currency turbulence

In [13]:
currency_hmm = fit_hmm(currency_turbulence.iloc[:,1].values)

Transition Probability =
[[0.9296 0.0704]
 [0.4023 0.5977]]
Mean =
[ 5.3518 21.6884]
Standard Deviation =
[ 3.6799 23.3688]
Initial Probability =
[1. 0.]


Similar to equity turbulence, the first regime in currency is normally distributed with mean 5.35 and standard deviation 3.67 while the second one has mean 21.69 and standard deviation 23.37. Their centers are volatilies are quite different than each other.

## Related Questions

### 1. What are the assumptions behind the model?

(a) We assume there are only two regimes in total.

(b) Each regime is normally distributed with certain mean and standard deviation.

(c) Every regime is independent of each other (correlation is 0).

### 2. What happens if they are not true?

### 3. What potential weaknesses and limitations can you identify in this approach?

(a) The algorithm is not robust to extremely large values (outlier). We previously ran our HMM without data cleansing. Some time series fail to converge and produce NaN due to these anomaly observations. In the future, we may consider using more advanced optimization algorithms that are robust.

## Reference

Kritzman, Mark, Sebastien Page, and David Turkington. "Regime shifts: Implications for dynamic strategies (corrected)." Financial Analysts Journal 68.3 (2012): 22-39.

Blackstone, Brian. "What Happened With the Swiss Franc?" The Wall Street Journal (2015). Retrieved from https://blogs.wsj.com/briefly/2015/01/15/what-happened-with-the-swiss-franc-the-short-answer/